# Set Up for Analysis

Tumblr Set Up

In [ ]:
!pip install pytumblr pyyaml rauth

In [ ]:
!pip install nltk
!pip install pandas
!pip install seaborn
!pip install beautifulsoup4

In [ ]:
import pytumblr
import os
import yaml
import webbrowser
from rauth import OAuth1Service
from pathlib import Path
from urllib.parse import urlparse, parse_qs

In [ ]:
import random
import re
import os
import time
import ast

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from collections import defaultdict
from datetime import datetime
from collections import Counter
from bs4 import BeautifulSoup

In [ ]:
# Set your Tumblr App credentials
consumer_key = "<consumer_key>"
consumer_secret = "<consumer_secret>"

# Token storage path
tumblr_token_path = Path.home() / ".tumblr"

# Function to load tokens from file
def load_tokens():
    if tumblr_token_path.exists():
        with open(tumblr_token_path, "r") as f:
            return yaml.safe_load(f)
    return None

# Function to save tokens to file
def save_tokens(tokens):
    with open(tumblr_token_path, "w") as f:
        yaml.dump(tokens, f)
    print(f"Tokens saved to {tumblr_token_path}")

In [ ]:
def do_oauth():
    tumblr = OAuth1Service(
        name='tumblr',
        consumer_key=consumer_key,
        consumer_secret=consumer_secret,
        request_token_url='https://www.tumblr.com/oauth/request_token',
        access_token_url='https://www.tumblr.com/oauth/access_token',
        authorize_url='https://www.tumblr.com/oauth/authorize',
        base_url='https://api.tumblr.com/v2/'
    )
    # Step 1: Get request token
    request_token, request_token_secret = tumblr.get_request_token(params={'oauth_callback': 'http://www.example.com'})
    authorize_url = tumblr.get_authorize_url(request_token) 

    print("Go to this URL and authorize the app:")
    print(authorize_url)

    webbrowser.open(authorize_url)

    verifier = input("Paste the verifier Tumblr shows you: ").strip()

    session = tumblr.get_auth_session(request_token, request_token_secret,
                                 method='POST', data={'oauth_verifier': verifier})
     # Print the oauth_token and oauth_token_secret
    print("OAuth Token:", session.access_token)
    print("OAuth Token Secret:", session.access_token_secret)

    return {
        'consumer_key': consumer_key,
        'consumer_secret': consumer_secret,
        'oauth_token': session.access_token,
        'oauth_token_secret': session.access_token_secret
    }

__After running the snippet below, and authorizing the app, the verifier is found in the URL after 'oauth_verifier='__

In [ ]:
tokens = do_oauth()

# Corpus Collection

In [ ]:
#Fill in these to make it easier to use again
consumer_key = '<consumer_key>'
consumer_secret = '<consumer_secret>'
oauth_token = '<oauth_token>'
oauth_token_secret = '<oauth_secret>'

In [ ]:
client = pytumblr.TumblrRestClient(
    '<consumer_key>',
    '<consumer_secret>',
    '<oauth_token>',
    '<oauth_secret>',
)

blog_name = '<replace with link to tumblr page>'

In [ ]:
client.info()

In [ ]:
import requests
import json
import pandas as pd

## Collecting Posts

### Create Character Tags and Variations (in bulk or individually)

Character tag creation in bulk

In [ ]:
#create list of characters 
characters = ['alma coin', 'annie cresta', 'asterid everdeen', 'atala', 'beetee latier', 'blight', 'boggs', 'bonnie', 'bristel', 
              'brutus', 'burdock everdeen', 'buttercup', 'caesar flickerman', 'cashmere', 'cassia', 'castor', 'cato hadley', 'cecelia', 'chaff', 'cinna', 
              'claudius templesmith',
              'clayton', 'clove', 'coriolanus snow', 'cray', 'cressida', 'darius', 'delly cartwright', 'effie trinket', 'enobaria', 
              'finnick odair', 'flavius', 'fulvia cardew', 'gale hawthorne', 'glimmer', 'gloss', 'goat man', 'greasy sae', 'haymitch abernathy',
              'hazelle hawthorne', 'homes', 'jackson', 'johanna mason', 'katniss everdeen', 'lady', 'lavinia', 'leevy', 'lyle whitcomb', 'lyme',
              'madge undersee', 'mags flanagan', 'martin', 'marvel', 'mayor undersee', 'messalla', 'miles', 'mitchell', 'morphlings', 'mrs mellark',
              'mrs. mellark', 'octavia', 'otho mellark', 'mr mellark', 'mrs everdeen', 'mr everdeen', 'otto', 'peeta mellark', 
              'plutarch heavensbee', 'pollux', 'portia', 'posy hawthorne', 'primrose everdeen', 'prim everdeen', 'purnia', 'paylor', 'ripper', 'romulus thread', 
              'rory hawthorne', 'rue', 'seeder', 'seneca crane', 'thresh', 'tigris snow', 'titus', 'twill', 'venia', 'wiress', 'woof', 
              'president snow', 'president coin'
             ]
#function to create print common variations for every character in the list above, accounting for character without last name with an else statement
#list for each character
def create_tags_per_character(list):
    for item in list: 
        if len(item.split()) > 1:
            first_name = item.split()[0]
            joined = item.replace(" ", "")
            print(f"['{first_name}', 'thg {first_name}', 'the hunger games {first_name}', 'hunger games {first_name}', '{first_name} thg', '{first_name} hg', 'hg {first_name}','{first_name} hunger games', ")
            print(f"'{joined}', 'thg {joined}', 'the hunger games {joined}', 'hunger games {joined}', '{joined} thg', '{joined} hg', 'hg {joined}','{joined} hunger games', ")
            print(f"'{item}', 'thg {item}', 'the hunger games {item}', 'hunger games {item}', '{item} thg', '{item} hg', 'hg {item}','{item} hunger games'] ---- ")
        else:
            print(f"['{item}', 'thg {item}', 'the hunger games {item}', 'hunger games {item}', '{item} thg', '{item} hg', 'hg {item}','{item} hunger games'] ---- ")

In [ ]:
#use this function to create tag variations for individual characters
def ch_tags(name):
    first_name = name.split()[0]
    joined = name.replace(" ", "")
    print(f"'{first_name}', 'thg {first_name}', 'the hunger games {first_name}', 'hunger games {first_name}', '{first_name} thg', '{first_name} hg', 'hg {first_name}','{first_name} hunger games', ")
    print(f"'{joined}', 'thg {joined}', 'the hunger games {joined}', 'hunger games {joined}', '{joined} thg', '{joined} hg', 'hg {joined}','{joined} hunger games', ")
    print(f"'{name}', 'thg {name}', 'the hunger games {name}', 'hunger games {name}', '{name} thg', '{name} hg', 'hg {name}','{name} hunger games', ")

In [ ]:
#Initial list of tags. adapt for your purposes if you choose not to use the functions above
tag_katniss = ['katniss everdeen', 'thg katniss', 'the hunger games katniss']
tag_peeta = ['peeta mellark', 'thg peeta', 'the hunger games peeta']
tag_everlark = ['everlark', 'katniss x peeta', 'peeta x katniss'] 
tag_gale = ['gale hawthorne', 'thg gale', 'the hunger games gale', 'gale']
tag_prim = ['primrose everdeen', 'thg prim', 'prim', 'the hunger games prim']
tag_haymitch = ['haymitch abernathy', 'thg haymitch', 'the hunger games haymitch', 'haymitch']
tag_effie = ['effie trinket', 'thg effie', 'the hunger games effie', 'effie']
tag_finnick = ['finnick odair', 'thg finnick', 'the hunger games finnick', 'finnick']
tag_beetee = ['beetee latier', 'thg beetee', 'the hunger games beetee', 'beetee']
tag_coin = ['alma coin', 'president coin']
tag_snow = ['coriolanus snow', 'president snow']
tag_ph = ['plutarch heavensbee', 'thg plutarch']
tag_rue = ['thg rue', 'the hunger games rue', 'rue']

In [ ]:
#list of tags excluded for various reasons. 
#I excluded fanfiction related ones - like non-canon relationships and tags related to writing
excluded_tags = []

In [ ]:
def del_duplicates(df):
    # Remove duplicates based on 'post_id' and keep the first occurrence
    df_cleaned = df.drop_duplicates(subset='post_id', keep='first').reset_index(drop=True)
    return df_cleaned

In [ ]:
#use this function to collect posts adjusting the parameters as needed
def collect_text_posts_for_tags(
    client,
    tags, #(list[str]): Tags to collect posts for
    excluded_tags=None, #(list[str]): Tags to exclude posts containing them
    max_posts_per_tag=1000, #Maximum number of posts per tag to collect.
    total_posts_to_collect=5000, #Total maximum posts to collect among all tags.
    posts_per_fetch=20 #Number of posts to fetch per API call (max 20 allowed by Tumblr)
):
    excluded_tags = excluded_tags or []
    normalized_excluded_tags = set(et.lower() for et in excluded_tags)
    collected_posts = []
    collected_post_ids = set()

    total_collected = 0

    for tag in tags:
        print(f"\nStarting collection for tag '{tag}'")

        normalized_tag = tag.lower()
        tag_collected = 0

        before = None  # Timestamp to paginate: fetch posts before this timestamp

        while tag_collected < max_posts_per_tag and total_collected < total_posts_to_collect:
            posts, oldest_ts = fetch_tagged_posts(client, tag, posts_per_fetch, before)

            if not posts:
                print(f"No more posts returned for tag '{tag}'. Ending this tag.")
                break

            new_posts_in_batch = 0

            for post in posts:
                # Check post type text
                if post.get('type') != 'text':
                    continue

                # Normalize post tags to lowercase for filtering
                post_tags = [t.lower() for t in post.get('tags', [])]

                # Check tag presence and exclusion tags
                if normalized_tag not in post_tags:
                    continue
                if any(ex_tag in post_tags for ex_tag in normalized_excluded_tags):
                    continue

                post_id = post.get('id')
                if post_id in collected_post_ids:
                    continue  # Duplicate post already collected

                # Prepare row data
                row = {
                    'post_id': post_id,
                    'date': post.get('date'),
                    'type': post.get('type'),
                    'text': post.get('body', post.get('summary', '')),
                    'tags': ', '.join(post.get('tags', []))
                }

                collected_posts.append(row)
                collected_post_ids.add(post_id)
                tag_collected += 1
                total_collected += 1
                new_posts_in_batch += 1

                print(f"Collected {total_collected} total posts; {tag_collected} posts for tag '{tag}'")

                if tag_collected >= max_posts_per_tag or total_collected >= total_posts_to_collect:
                    break

            if new_posts_in_batch == 0:
                print(f"No new valid posts in this fetch for tag '{tag}'. Ending tag early to avoid endless loop.")
                break

            before = oldest_ts  # Use oldest post timestamp for next pagination
            time.sleep(2)  # Be gentle to API rate limits

    print("\nCollection complete. Deduplicating posts...")

    # Create DataFrame and remove duplicates by post_id keeping first occurrence
    df = pd.DataFrame(collected_posts)
    df = df.drop_duplicates(subset='post_id', keep='first').reset_index(drop=True)

    print(f"Total unique text posts collected: {len(df)}")

    # Save dataframe as CSV with tag list name in filename
    tags_filename_part = "_".join([t.replace(" ", "_") for t in tags])
    csv_filename = f'tumblr_text_posts_{tags_filename_part}.csv'
    df.to_csv(csv_filename, index=False)
    print(f"Data saved to CSV file: {csv_filename}")

    return df

In [ ]:
#run for every relevant character
collect_text_posts_for_tags(
    client,
    tags, #(list[str]): Tags to collect posts for
    excluded_tags=None, #(list[str]): Tags to exclude posts containing them
    max_posts_per_tag=1000, #Maximum number of posts per tag to collect.
    total_posts_to_collect=5000, #Total maximum posts to collect among all tags.
    posts_per_fetch=20 #Number of posts to fetch per API call (max 20 allowed by Tumblr)
)

# Tag Frequency Analysis

## Preprocess for tag-frequency-analysis and text analysis

In [ ]:
#load dataframes of collected posts
character_df = pd.read_csv("<insert .csv files saved from previous step")
#repeat as many times as needed
#eg: katniss_df = pd.read_csv("tumblr_text_posts_katniss_everdeen_thg_katniss_the_hunger_games_katniss.csv")

Use the functions below as needed

In [ ]:
def norm_tags(tags):
    tag_list = []
    for tag in tags:
        tag = tag.lower()
        # Remove non-ASCII characters and emojis
        tag = re.sub(r'[^\x00-\x7F]+', '', tag)  # Remove non-ASCII characters
        tag_list.append(tag)     
    return tag_list
    
def tag_counter(df):
    # Clean the tags column to remove non-ASCII characters before splitting
    df['tags'] = df['tags'].apply(lambda x: re.sub(r'[^\x00-\x7F]+', '', x) if isinstance(x, str) else x)
    
    # Split the tags by comma and strip whitespace
    df['tags'] = df['tags'].apply(lambda x: [tag.strip() for tag in x.split(',')] if isinstance(x, str) else x)
    
    # Clean tags immediately after conversion
    df['tags'] = df['tags'].apply(lambda tags: [re.sub(r'[^\x00-\x7F]+', '', tag.lower()) for tag in tags])
    
    df_rel_tags = [tag for tags in df['tags'] for tag in tags]
    df_taglist = norm_tags(df_rel_tags)
    df_tag_counts = Counter(df_taglist)
    return df_tag_counts

def print_low_freq_tags(df, threshold):
    # Get the tag counts
    tag_counts = tag_counter(df)
    # Filter tags with frequency less than the threshold
    low_freq_tags = [tag for tag, count in tag_counts.items() if count < threshold]
    # Print the low frequency tags
    return low_freq_tags

Use __print_low_freq_tags(df, threshold)__ to further filter tags and add to excluded tags list.
Repeat with every dataframe if required/desired.

In [ ]:
ch_freq = tag_counter(character_df)
ch_freq

In [ ]:
print_low_freq_tags(character_df, 5)

In [ ]:
#add tags to exclude/dataframe to excluded_tags, or individual lists per dataframe
ch_excl_tags = []

In [ ]:
#if done individually, combine them all
exclude_all = ch_excl_tags1 + ch_excl_tags2 + ch_excl_tags3
masterlist = list(set(exclude_all))
masterlist = sorted(masterlist)

In [ ]:
#copy output of this cell into excluded_tags list
print(masterlist)

In [ ]:
#repeat collection for final dataframe per character 
collect_text_posts_for_tags(
    client,
    tags, #(list[str]): Tags to collect posts for
    excluded_tags=None, #(list[str]): Tags to exclude posts containing them
    max_posts_per_tag=1000, #Maximum number of posts per tag to collect.
    total_posts_to_collect=5000, #Total maximum posts to collect among all tags.
    posts_per_fetch=20 #Number of posts to fetch per API call (max 20 allowed by Tumblr)
)

## Tag Frequency Analysis

In [ ]:
df

In [ ]:
char_tags = char_df['tags']

In [ ]:
#creates dictionary of tags and frequency
all_char_tags = char_df['tags'].str.cat(sep=', ').split(', ')
char_tags_count = Counter(all_char_tags)

In [ ]:
#list tags that are unrelated (eg: character from another fictional universe, etc.)
remove_tags = [] 

In [ ]:
#removing unrelated posts using tags
char_df = char_df[~char_df['tags'].str.contains('|'.join(remove_tags), case=False, na=False)]

In [ ]:
#preprocess text columns
char_df['text'] = char_df['text'].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text() if pd.notnull(x) else '')
char_df['text'] = char_df['text'].str.replace('\n', ' ', regex=False).str.replace('\r', ' ', regex=False)

In [ ]:
char_df.to_csv('char_processed_df.csv', index=False)

# Preparing for using Riveter

In [ ]:
#for riveter
!pip install -U spacy-experimental
!pip install https://github.com/explosion/spacy-experimental/releases/download/v0.6.0/en_coreference_web_trf-3.4.0a0-py3-none-any.whl
#egg=en_coreference_web_trf
!spacy download en_core_web_sm

In [ ]:
pwd #to confirm directory

In [ ]:
!pip install -e .

In [ ]:
#just in case it was missed earlier
from collections import defaultdict
from collections import Counter
from datetime import datetime
import pandas as pd
import random
import numpy as np
import requests
import re
import ast
import glob

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# SPACY & COREF IMPORTS
import spacy
import spacy_experimental
nlp = spacy.load("en_core_web_sm")
nlp_coref = spacy.load("en_coreference_web_trf")

nlp_coref.replace_listeners("transformer", "coref", ["model.tok2vec"])
nlp_coref.replace_listeners("transformer", "span_resolver", ["model.tok2vec"])

nlp.add_pipe("coref", source=nlp_coref)
nlp.add_pipe("span_resolver", source=nlp_coref)

In [ ]:
from riveter import Riveter

In [ ]:
#for preprocessing text
from bs4 import BeautifulSoup
import html

In [ ]:
#load dataframes
p2_dfs = glob.glob("*processed_df.csv")

In [ ]:
p2dfs

In [ ]:
def convert(file):
    file_df = pd.read_csv(file)
    return file_df

In [ ]:
#combine all desired dfs to create corpus for riveter analysis
p2_char1_df = convert('char1_processed_df.csv')
p2_char2_df = convert('char2_processed_df.csv')
p2_char3_df = convert('char3_processed_df.csv')

In [ ]:
all_dfs = [p2_char1_df, p2_char2_df, p2_char3_df]
p2_corpus = pd.concat(all_dfs)

In [ ]:
p2_corpus

In [ ]:
phase_2_corpus = del_duplicates(p2_corpus) #remove duplicates

In [ ]:
phase_2_corpus

## Preprocess of Riveter

In [ ]:
keep_columns = ['post_id', 'text']

In [ ]:
#convert dicts to a list of values from the dictionary
def dict_val_to_list(my_dict):
  if not isinstance(my_dict, dict):
    print("Error: Input is not a dictionary.")
    return []  # Return an empty list if the input is not a dictionary

  return list(my_dict.values())

In [ ]:
def parse_html_list(html_list):
    cleaned = []
    for item in html_list:
        if not isinstance(item, str):
            # Option 1: convert to string
            item = str(item)
            # Option 2: skip non-string items by uncommenting the next line
            # continue
        soup = BeautifulSoup(item, 'html.parser')
        text = soup.get_text(separator=' ', strip=True)
        text = re.sub(r'\xa0', '', text)
        cleaned.append(text)
    return cleaned

In [ ]:
p2 = phase_2_corpus[keep_columns]
p2_id_dict = p2['post_id'].to_dict()
p2_ids = dict_val_to_list(p2_id_dict)
p2_body_dict = p2['text'].to_dict()
p2_text = dict_val_to_list(p2_body_dict)

In [ ]:
p2_texts = parse_html_list(p2_text)

In [ ]:
#Use this dict like an alias map - use regex so model is trained to recognize different ways of naming a character
persona_patterns_dict={'katniss': r'^katniss$|^katniss everdeen$|^girl on fire$',
                        'peeta': r'^peeta$|^peeta mellark$|^boy with the bread$',
                        'haymitch': r'^haymitch$|^haymitch abernathy$',
                        'president snow': r'^coryo$|^snow$|^president snow$|^coriolanus snow$',
                        'president coin': r'^coin$|^alma coin$|^president coin$|^d13 president$',
                        'primrose': r'^prim$|^primrose$|^primrose everdeen$',
                        'gale': r'^gale$|^gale hawthorne$', 
                        'finnick': r'^finnick$|^finnick odair$',
                        'plutarch': r'^plutarch$|^plutarch heavensbee$',
                        'annie': r'^annie$|^annie cresta$',
                        'johanna': r'^johanna$|^johanna mason$',
                        'rue': r'^rue$',
                        'effie': r'^effie$|^effie trinket$',
                        'capitol': r'^the capitol$|^capitol$|^the rich$|^the wealthy$|^government$',
                       }

# Use Riveter - Computational Linguistic and Sentiment Analysis

Warning: Training models can take an hour or more depending on the size of the dataset

## Sap et al's power and agency(Linguistic) 

In [ ]:
riveter = Riveter()
riveter.load_sap_lexicon('agency')
riveter.train(p2_texts,
              p2_ids,
              persona_patterns_dict={'katniss': r'^katniss$|^katniss everdeen$|^girl on fire$',
                        'peeta': r'^peeta$|^peeta mellark$|^boy with the bread$',
                        'haymitch': r'^haymitch$|^haymitch abernathy$',
                        'president snow': r'^coryo$|^snow$|^president snow$|^coriolanus snow$',
                        'president coin': r'^coin$|^alma coin$|^president coin$|^d13 president$',
                        'primrose': r'^prim$|^primrose$|^primrose everdeen$',
                        'gale': r'^gale$|^gale hawthorne$', 
                        'finnick': r'^finnick$|^finnick odair$',
                        'plutarch': r'^plutarch$|^plutarch heavensbee$',
                        'annie': r'^annie$|^annie cresta$',
                        'johanna': r'^johanna$|^johanna mason$',
                        'rue': r'^rue$',
                        'effie': r'^effie$|^effie trinket$',
                        'capitol': r'^the capitol$|^capitol$|^the rich$|^the wealthy$|^government$',
                       }
             )

Consult demo.ipynb or myriv-test.ipynb to see options for different tools and analyses options

## Rashkin et. al's Connotation Frames (Sentiment)

#### Connotation Frames available via Rashkin: 
- __effect__: whether the event denoted by a predicate is good or bad for the entity
- __state__: the likely mental state of an entity as the result of an event
- __value__: whether an entity is presupposed to be valuable
- __writer_perspective__/__reader_perspective__: the directed sentiment from the writer to an entity or the _predicted_ directed sentiment from reader to an entity
- __agent_theme_perspective__/__theme_agent_perspective__: the directed sentiment between the agent and theme (usually reciprocal and not likely to totally contradict each other). 

Connotation frame polarity can be positive, negative, or neutral. 


In [ ]:
riveter = Riveter()
riveter.load_sap_lexicon('dimension')
riveter.train(p2_texts,
              p2_ids,
              persona_patterns_dict={'katniss': r'^katniss$|^katniss everdeen$|^girl on fire$',
                        'peeta': r'^peeta$|^peeta mellark$|^boy with the bread$',
                        'haymitch': r'^haymitch$|^haymitch abernathy$',
                        'president snow': r'^coryo$|^snow$|^president snow$|^coriolanus snow$',
                        'president coin': r'^coin$|^alma coin$|^president coin$|^d13 president$',
                        'primrose': r'^prim$|^primrose$|^primrose everdeen$',
                        'gale': r'^gale$|^gale hawthorne$', 
                        'finnick': r'^finnick$|^finnick odair$',
                        'plutarch': r'^plutarch$|^plutarch heavensbee$',
                        'annie': r'^annie$|^annie cresta$',
                        'johanna': r'^johanna$|^johanna mason$',
                        'rue': r'^rue$',
                        'effie': r'^effie$|^effie trinket$',
                        'capitol': r'^the capitol$|^capitol$|^the rich$|^the wealthy$|^government$',
                       }
             )